# MNIST Classifier with HuoguoML

This short notebook shows how to use HuoguoML to deploy and manage a service. This includes:

1. Building and training a neural network to classify MNIST images
2. Create a HuoguoML service

## Requirements

We will be using Tensorflow 2 and HuoguoML. Update `pip` and install packages:

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install huoguoml

Import all packages:

In [ ]:
import tensorflow as tf

## Building and training a neural network to classify MNIST images

### Data

First we load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Further, the samples are converted from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Create a neural network
A two layer fully connected neural network, that is trained with cross entropy loss and optimized with adam model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train, evaluate and export neural network

Train neural network

In [ ]:
model.fit(x_train, y_train, epochs=5)

Evaluate neural network -the image classifier should have a accuracy of ~98% on this dataset.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

Export neural network

In [ ]:
model_dir = "./model"
model.save(model_dir)

In [ ]:
!saved_model_cli show --dir ./model --all

## Create a HuoguoML service

In [1]:
import huoguoml

In [4]:
huoguoml.create_experiment("mnist", "./huoguoml")
huoguoml.experiment

Experiment(id=1, name='mnist')

In [7]:
huoguoml.tensorflow.log_model(tf_saved_model_dir="./model", 
                              tf_meta_graph_tags="serve",
                              tf_signature_def_key="serving_default")